In [1]:
import cv2
import numpy as np

def stitch_images(image1_path, image2_path, ratio_threshold=0.75, ransac_threshold=5.0):
    """
    Stitches two images together to create a panoramic image using feature matching and homography.

    Parameters:
    - image1_path: Path to the first image.
    - image2_path: Path to the second image.
    - ratio_threshold: Ratio test threshold for feature matching (default is 0.75).
    - ransac_threshold: RANSAC pixel threshold for homography estimation (default is 5.0).

    Returns:
    - result: The stitched panorama image.
    """
    # Load the images
    img1 = cv2.imread(image1_path)
    img2 = cv2.imread(image2_path)
    
    # Convert to grayscale
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    
    # Detect and compute features using ORB
    orb = cv2.ORB_create()
    keypoints1, descriptors1 = orb.detectAndCompute(gray1, None)
    keypoints2, descriptors2 = orb.detectAndCompute(gray2, None)
    
    # Match descriptors using BFMatcher with L2 norm
    bf = cv2.BFMatcher(cv2.NORM_L2)
    matches = bf.knnMatch(descriptors1, descriptors2, k=2)
    
    # Apply the ratio test to filter matches
    good_matches = []
    for m, n in matches:
        if m.distance < ratio_threshold * n.distance:
            good_matches.append(m)
    
    if len(good_matches) < 4:
        print("Not enough good matches to estimate homography.")
        return None
    
    # Extract matched points
    src_points = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_points = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    
    # Estimate the homography matrix using RANSAC
    homography_matrix, mask = cv2.findHomography(src_points, dst_points, cv2.RANSAC, ransac_threshold)
    
    # Warp one image onto the other using the homography matrix
    height, width, _ = img2.shape
    panorama = cv2.warpPerspective(img1, homography_matrix, (width + img1.shape[1], height))
    
    # Overlay the second image onto the panorama
    panorama[0:height, 0:width] = img2
    
    return panorama

if __name__ == "__main__":
    # Paths to the input images
    image1_path = "/home/student/Pictures/Screenshots/Screenshot from 2024-12-06 11-15-29.png"  # Replace with your image path
    image2_path = "/home/student/Pictures/Screenshots/Screenshot from 2024-12-06 11-30-13.png"  # Replace with your image path

    # Stitch the images together
    panorama = stitch_images(image1_path, image2_path)
    
    if panorama is not None:
        # Display the stitched panorama
        cv2.imshow("Panorama", panorama)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
        # Save the result
        cv2.imwrite("panorama.jpg", panorama)
        print("Panorama saved as 'panorama.jpg'")
    else:
        print("Failed to stitch images.")


QSocketNotifier: Can only be used with threads started with QThread


Panorama saved as 'panorama.jpg'
